In [2]:
import os
%pwd

'/home/towet/Desktop/Visions/OpenProjects/Lung-Colon-Cancer-Histopathological-Images-Classification-Project/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/home/towet/Desktop/Visions/OpenProjects/Lung-Colon-Cancer-Histopathological-Images-Classification-Project'

In [ ]:
os.eviron["MLFLOW_TRACKING_URI"]="https://dagshub.com/Towet-Tum/Lung-Colon-Cancer-Histopathological-Images-Classification-Project.mlflow "
os.environ["MLFLOW_TRACKING_USERNAME"]="Towet-Tum"
os.environ["MLFLOW_TRACKING_PASSWORD"]="39c66009c3d7d6d659cbc691e65a93c46873d3c4"


In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    dataset: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [6]:
from LungColonClassifier.constants import *
from LungColonClassifier.utils.common import read_yaml, create_directories, save_json

In [7]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        dataset = os.path.join("artifacts", "data_ingestion", "lung_colon_image_set")
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/xception_model.h5",
            mlflow_uri="https://dagshub.com/Towet-Tum/Lung-Colon-Cancer-Histopathological-Images-Classification-Project.mlflow ",
            all_params=self.params,
            params_image_size=self.params.IMG_SZ,
            params_batch_size=self.params.BATCH_SIZE,
            dataset=Path(dataset)
        )
        return eval_config


In [8]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse
from LungColonClassifier.components.model_trainer import Training
from LungColonClassifier.config.configuration import ConfigurationManager

2024-02-01 19:29:18.127450: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-01 19:29:18.183726: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-01 19:29:18.183783: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-01 19:29:18.185286: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-01 19:29:18.194356: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-01 19:29:27.032455: W tensorflow/compiler/tf2tensorrt/utils/py_utils.

In [9]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config



    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        obj = ConfigurationManager()
        obj_conf = obj.get_training_config()
        train = Training(config=obj_conf)
        df = train.load_data(self.config.dataset)
        _, _, test_gen = train.process_data(df)

        self.model = self.load_model(self.config.path_of_model)
        
        self.score = self.model.evaluate(test_gen)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                mlflow.keras.log_model(self.model, "model", registered_model_name="XceptionModel")
            else:
                mlflow.keras.log_model(self.model, "model")

In [10]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-02-01 19:30:21,420: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-02-01 19:30:21,428: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-01 19:30:21,434: INFO: common: created directory at: artifacts]


AttributeError: 'ConfigurationManager' object has no attribute 'get_evaluation_config'